**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer 1__:

```epsilon``` is the probability to explore a random state. If epsilon is too small, the agent would never explore the environment. However, if epsilon is too big the agent would have a harder time to learn anything new and will explore randomly too often.

***

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        # ??????????????????????
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=3 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__ANSWER 2__

```board``` : corresponds to the board where the rat moves. It is initialised randomly and contains the reward for cheese and poison. When the mouse passes by a case, the reward corresponding is set to 0 as the object was eaten.

```position``` : corresponds to the position of the rat on the board. There, states that are innacessible (out of the board are set to -1) the rest is 0, this allows the rat to see if it is close to a boundarie or not.

***

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return(np.random.randint(4)) # select a random action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    over= False
    for e in range(epochs):
        win=0
        lose=0
        s=env.reset()
        while not over:
            a = agent.learned_act(s)
            s,r,over = env.act(a)
            if(r>0):
                win+=r
            else:
                lose-=r
        # Save as a mp4
        env.draw(prefix+str(e))
        # Update stats
        score = score + win-lose
        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,10,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/11.0. Average score (-2.0)
Win/lose count 0/0. Average score (-1.0)
Win/lose count 0/0. Average score (-0.6666666666666666)
Win/lose count 0/0. Average score (-0.5)
Win/lose count 0/0. Average score (-0.4)
Win/lose count 0/0. Average score (-0.3333333333333333)
Win/lose count 0/0. Average score (-0.2857142857142857)
Win/lose count 0/0. Average score (-0.25)
Win/lose count 0/0. Average score (-0.2222222222222222)
Win/lose count 0/0. Average score (-0.2)
Final score: -0.2


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




# ANSWER

\begin{align}
Q^{\pi}(s,a) &= E\left[ \sum_{t \geq 0} \gamma^t r(s_t,a_t) | s_0,a_0=a,s, \forall t>0, a_t = \pi(s_t)\right] \\
&= \gamma * E\left[ \sum_{t \geq 0} \gamma^t r(s_t,a_t) | s_0 \sim p(. |a,s), \forall t>0, a_t = \pi(s_t)\right] + r(s,a) \\
&= \gamma * E_{s_0 \sim p(. |a,s)}\left[ \sum_{t \geq 0} \gamma^t r(s_t,a_t) |\forall t>0, a_t = \pi(s_t)\right] + r(s,a), \text{we recognize Q inside the esperance} \\
&= E_{s' \sim p(. |a,s)}\left[ r(s,a) + \gamma Q(s',\pi(s'))   \right]
\end{align}

Then knowing that : $Q^*(s,a) = \max_{\pi}{Q^{\pi}(s,a)} \forall s,a $

\begin{align}
Q^{\pi}(s,a) &= \max_{\pi} E_{s' \sim p(. |a,s)}\left[ r(s,a) + \gamma Q(s',\pi(s'))   \right] \\
 &=  r(s,a) + \max_{\pi} \sum_{s'}{ P(s'=s'|a,s) + \gamma Q(s'',\pi(s'')) } \\
 &=  r(s,a) +  \sum_{s'}{ P(s''=s'|a,s) + \gamma \max_{\pi} Q(s'',\pi(s'')) } \\
 &=  r(s,a) +  \sum_{s'}{ P(s''=s'|a,s) + \gamma \max_{a} Q*(s'',a') } \\
 &= \max_{\pi} E_{s' \sim p(. |a,s)}\left[ r(s,a) + \gamma \max_{a} Q(s',a') \right] \\
\end{align}

Finally our model $Q(.,.,\theta)$ is close to $Q^*$ then

\begin{align}
Q(s,a, \theta) = Q^*(s,a, \theta) &\iff E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \\
&\iff E_{s' \sim p(.|s,a)} [r +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)]=0\\
\end{align}
To approximate $Q^*$ we can then minize $$\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        
    def remember(self, m):
        if(len(self.memory)>=self.max_memory):
            del(self.memory[0])
        self.memory.append(m)

    def random_access(self):
        return( self.memory[np.random.randint(len(self.memory))])

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
epochs_train=3 # set small when debugging
epochs_test=1 # set small when debugging
from keras.layers import Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #print(s[0,:,0])
        s = s.reshape(1,5,5,self.n_state)
        #print(s[0,0,:,0])
        return(np.argmax(self.model.predict(s).ravel()))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, ns, a,r,game_over = self.memory.random_access()
            s = s.reshape(1,5,5,self.n_state)
            ns = ns.reshape(1,5,5,self.n_state)
            input_states[i, :, :,:] = s
            target_q[i,:] = self.model.predict(s)
            if game_over:
                target_q[i,a] = r
            else:
                target_q[i,a] = r + self.discount * np.max(self.model.predict(ns))
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential([
            Flatten(input_shape=(5,5,self.n_state)),
            Dense(20, activation='relu'),
            Dense(4),
        ])
        print(model.summary())
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model

In [12]:
epochs_train = 201
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 84        
Total params: 1,104
Trainable params: 1,104
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/201 | Loss 0.0043 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/201 | Loss 0.0083 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/201 | Loss 0.0054 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/201 | Loss 0.0055 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/201 | Loss 0.0086 | Win/lose count 2.0/4.0 (-2.0)
Epoch 005/201 | Loss 0.0110 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/201 | Loss 0.0163 | Win/lose co

Epoch 127/201 | Loss 0.0036 | Win/lose count 5.5/1.0 (4.5)
Epoch 128/201 | Loss 0.0472 | Win/lose count 18.5/3.0 (15.5)
Epoch 129/201 | Loss 0.0027 | Win/lose count 4.0/1.0 (3.0)
Epoch 130/201 | Loss 0.0028 | Win/lose count 15.0/4.0 (11.0)
Epoch 131/201 | Loss 0.0027 | Win/lose count 13.5/3.0 (10.5)
Epoch 132/201 | Loss 0.0043 | Win/lose count 23.5/3.0 (20.5)
Epoch 133/201 | Loss 0.0042 | Win/lose count 15.5/2.0 (13.5)
Epoch 134/201 | Loss 0.0027 | Win/lose count 9.0/3.0 (6.0)
Epoch 135/201 | Loss 0.0038 | Win/lose count 15.0/0 (15.0)
Epoch 136/201 | Loss 0.0033 | Win/lose count 13.0/2.0 (11.0)
Epoch 137/201 | Loss 0.0054 | Win/lose count 19.5/4.0 (15.5)
Epoch 138/201 | Loss 0.0042 | Win/lose count 11.5/3.0 (8.5)
Epoch 139/201 | Loss 0.0057 | Win/lose count 10.0/2.0 (8.0)
Epoch 140/201 | Loss 0.0039 | Win/lose count 8.0/1.0 (7.0)
Epoch 141/201 | Loss 0.0035 | Win/lose count 11.5/5.0 (6.5)
Epoch 142/201 | Loss 0.0035 | Win/lose count 25.0/2.0 (23.0)
Epoch 143/201 | Loss 0.0033 | Win/los

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [12]:
from keras.layers import Conv2D, SeparableConv2D

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential([
            Conv2D(64, kernel_size=3, input_shape=(5,5,self.n_state), activation='relu'),
            Conv2D(16, kernel_size=2, strides=(1,1), activation='relu'),
            Flatten(),
            Dense(4),
        ])
        print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
epochs_train = 301
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 64)          1216      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 16)          4112      
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 5,588
Trainable params: 5,588
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/301 | Loss 0.0064 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/301 | Loss 0.0041 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/301 | Loss 0.0105 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/301 | Loss 0.0009 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/301 | Loss 0.0087 | 

Epoch 122/301 | Loss 0.0026 | Win/lose count 15.5/6.0 (9.5)
Epoch 123/301 | Loss 0.0015 | Win/lose count 17.0/6.0 (11.0)
Epoch 124/301 | Loss 0.0042 | Win/lose count 14.5/6.0 (8.5)
Epoch 125/301 | Loss 0.0012 | Win/lose count 8.0/4.0 (4.0)
Epoch 126/301 | Loss 0.0483 | Win/lose count 18.5/7.0 (11.5)
Epoch 127/301 | Loss 0.0014 | Win/lose count 17.5/1.0 (16.5)
Epoch 128/301 | Loss 0.0475 | Win/lose count 21.5/2.0 (19.5)
Epoch 129/301 | Loss 0.0010 | Win/lose count 25.5/5.0 (20.5)
Epoch 130/301 | Loss 0.0032 | Win/lose count 14.0/8.0 (6.0)
Epoch 131/301 | Loss 0.0044 | Win/lose count 17.0/6.0 (11.0)
Epoch 132/301 | Loss 0.0022 | Win/lose count 14.5/2.0 (12.5)
Epoch 133/301 | Loss 0.0023 | Win/lose count 18.5/2.0 (16.5)
Epoch 134/301 | Loss 0.0015 | Win/lose count 17.0/6.0 (11.0)
Epoch 135/301 | Loss 0.0027 | Win/lose count 12.5/4.0 (8.5)
Epoch 136/301 | Loss 0.0015 | Win/lose count 24.0/4.0 (20.0)
Epoch 137/301 | Loss 0.0011 | Win/lose count 20.0/5.0 (15.0)
Epoch 138/301 | Loss 0.0499 | 

Epoch 258/301 | Loss 0.0021 | Win/lose count 30.5/5.0 (25.5)
Epoch 259/301 | Loss 0.0018 | Win/lose count 15.0/7.0 (8.0)
Epoch 260/301 | Loss 0.0019 | Win/lose count 13.0/4.0 (9.0)
Epoch 261/301 | Loss 0.0016 | Win/lose count 22.5/5.0 (17.5)
Epoch 262/301 | Loss 0.0012 | Win/lose count 20.5/4.0 (16.5)
Epoch 263/301 | Loss 0.0017 | Win/lose count 13.0/3.0 (10.0)
Epoch 264/301 | Loss 0.0010 | Win/lose count 18.5/6.0 (12.5)
Epoch 265/301 | Loss 0.0007 | Win/lose count 22.0/4.0 (18.0)
Epoch 266/301 | Loss 0.0011 | Win/lose count 19.0/7.0 (12.0)
Epoch 267/301 | Loss 0.0531 | Win/lose count 11.0/4.0 (7.0)
Epoch 268/301 | Loss 0.0022 | Win/lose count 16.0/4.0 (12.0)
Epoch 269/301 | Loss 0.0017 | Win/lose count 14.0/7.0 (7.0)
Epoch 270/301 | Loss 0.0013 | Win/lose count 17.0/4.0 (13.0)
Epoch 271/301 | Loss 0.0037 | Win/lose count 12.0/4.0 (8.0)
Epoch 272/301 | Loss 0.0024 | Win/lose count 20.5/6.0 (14.5)
Epoch 273/301 | Loss 0.0542 | Win/lose count 19.5/7.0 (12.5)
Epoch 274/301 | Loss 0.0509 |

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [13]:
epochs_test=11
env = Environment(grid_size=size, max_time=T,temperature=0.1) # temperature lower to see if the algo manage to explore
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 64)          1216      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 16)          4112      
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 5,588
Trainable params: 5,588
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 50)                0         
_________________________________________________________________
den

In [16]:
HTML(display_videos('cnn_test0.mp4'))

In [17]:
HTML(display_videos('fc_test0.mp4'))

### Problems :

When there is no cheese, the algorithme is a bit lost when there is no cheese. The algorithme has hard time exploring and going to new era to consume new cheese when all has been consumed.

Then when the agent has learnt a lot, it will still randomly visit states sometimes eventhough exploring is no used when it is not training anymore.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [30]:
## use those samples of code:
#In train explore:

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16
        
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action, train):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        if train:
            reward += self.malus_position[self.x, self.y]
        
        self.malus_position[self.x, self.y] = - 0.1
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
    
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [32]:
def train_explore(agent,env,epoch,prefix='', decay = 0.5):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        win = 0
        lose = 0


        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        if e%20==0:
            agent.set_epsilon(agent.epsilon*0.7)

            # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')    

In [34]:
# Training
epochs_train=301
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 3, 3, 64)          1792      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 16)          4112      
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 260       
Total params: 6,164
Trainable params: 6,164
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/301 | Loss 0.0179 | Win/lose count 4.5/26.000000000000092 (-21.500000000000092)
Epoch 001/301 | Loss 0.0094 | Win/lose count 9.0/23.900000000000055 (-14.900000000000055)
Epoch 002/301 | Loss 0.0068 | Win/lose count 9.0/21.300000000000026 (-12.30000000000002

Epoch 083/301 | Loss 0.0093 | Win/lose count 15.5/17.79999999999999 (-2.29999999999999)
Epoch 084/301 | Loss 0.0055 | Win/lose count 22.5/13.999999999999973 (8.500000000000027)
Epoch 085/301 | Loss 0.0152 | Win/lose count 26.5/10.999999999999977 (15.500000000000023)
Epoch 086/301 | Loss 0.0073 | Win/lose count 29.0/10.999999999999982 (18.000000000000018)
Epoch 087/301 | Loss 0.0190 | Win/lose count 26.0/16.599999999999973 (9.400000000000027)
Epoch 088/301 | Loss 0.0090 | Win/lose count 23.0/12.999999999999973 (10.000000000000027)
Epoch 089/301 | Loss 0.0103 | Win/lose count 20.0/12.59999999999997 (7.4000000000000306)
Epoch 090/301 | Loss 0.0069 | Win/lose count 27.0/13.699999999999978 (13.300000000000022)
Epoch 091/301 | Loss 0.0090 | Win/lose count 15.0/13.799999999999967 (1.200000000000033)
Epoch 092/301 | Loss 0.0086 | Win/lose count 22.0/12.79999999999998 (9.20000000000002)
Epoch 093/301 | Loss 0.0144 | Win/lose count 22.5/13.599999999999975 (8.900000000000025)
Epoch 094/301 | Loss

Epoch 175/301 | Loss 0.0126 | Win/lose count 19.5/9.99999999999998 (9.50000000000002)
Epoch 176/301 | Loss 0.0118 | Win/lose count 25.5/12.799999999999978 (12.700000000000022)
Epoch 177/301 | Loss 0.0095 | Win/lose count 20.0/12.399999999999977 (7.600000000000023)
Epoch 178/301 | Loss 0.0233 | Win/lose count 26.0/12.299999999999976 (13.700000000000024)
Epoch 179/301 | Loss 0.0147 | Win/lose count 25.0/13.699999999999973 (11.300000000000027)
Epoch 180/301 | Loss 0.0247 | Win/lose count 24.0/13.099999999999978 (10.900000000000022)
Epoch 181/301 | Loss 0.0113 | Win/lose count 10.5/15.599999999999964 (-5.099999999999964)
Epoch 182/301 | Loss 0.0232 | Win/lose count 25.0/14.999999999999973 (10.000000000000027)
Epoch 183/301 | Loss 0.0096 | Win/lose count 15.5/13.799999999999969 (1.7000000000000313)
Epoch 184/301 | Loss 0.0131 | Win/lose count 23.0/11.199999999999978 (11.800000000000022)
Epoch 185/301 | Loss 0.0161 | Win/lose count 24.5/10.399999999999979 (14.100000000000021)
Epoch 186/301 |

Epoch 267/301 | Loss 0.0341 | Win/lose count 21.0/12.599999999999975 (8.400000000000025)
Epoch 268/301 | Loss 0.0267 | Win/lose count 19.0/12.299999999999972 (6.700000000000028)
Epoch 269/301 | Loss 0.0390 | Win/lose count 15.0/12.89999999999997 (2.10000000000003)
Epoch 270/301 | Loss 0.0366 | Win/lose count 17.0/12.099999999999973 (4.900000000000027)
Epoch 271/301 | Loss 0.0110 | Win/lose count 14.0/16.09999999999996 (-2.099999999999959)
Epoch 272/301 | Loss 0.0216 | Win/lose count 17.0/14.699999999999976 (2.300000000000024)
Epoch 273/301 | Loss 0.0197 | Win/lose count 25.5/14.899999999999968 (10.600000000000032)
Epoch 274/301 | Loss 0.0105 | Win/lose count 23.5/12.599999999999978 (10.900000000000022)
Epoch 275/301 | Loss 0.0216 | Win/lose count 19.5/12.69999999999997 (6.800000000000029)
Epoch 276/301 | Loss 0.0277 | Win/lose count 22.0/14.499999999999964 (7.5000000000000355)
Epoch 277/301 | Loss 0.0135 | Win/lose count 26.5/11.299999999999976 (15.200000000000024)
Epoch 278/301 | Loss

In [37]:
# Evaluation
def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    over= False
    s=env.reset()
    for e in range(epochs):
        win=0
        lose=0
        s=env.reset()
        over = False
        while not over:
            a = agent.learned_act(s)
            s,r,over = env.act(a, train=False)
            if(r>0):
                win+=r
            else:
                lose-=r
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
        # Update stats
        score = score + win-lose
        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

epochs_test=11
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test_explore(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 8.0/1.0. Average score (7.0)
Win/lose count 22.0/6.0. Average score (11.5)
Win/lose count 17.5/1.0. Average score (13.166666666666666)
Win/lose count 16.5/0.0. Average score (14.0)
Win/lose count 28.0/4.0. Average score (16.0)
Win/lose count 19.0/4.0. Average score (15.833333333333334)
Win/lose count 25.5/2.0. Average score (16.928571428571427)
Win/lose count 20.0/2.0. Average score (17.0625)
Win/lose count 25.0/3.0. Average score (17.61111111111111)
Win/lose count 24.0/2.0. Average score (18.05)
Win/lose count 17.5/1.0. Average score (17.90909090909091)
Final score: 17.90909090909091


This model perform way better than the last one : it doesn't get stuck anywhere (even when its espilon is really small at the end of the training). However it still passes through poison cases to explore.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***